In [168]:
# !kaggle competitions download -c dog-breed-identification

In [169]:
# !unzip dog-breed-identification.zip -d ./dataset/
# !rm dog-breed-identification.zip

In [170]:
import pandas as pd
from torch.utils.data import Dataset, DataLoader, random_split
import torch.nn  as nn
import os
from PIL import Image
from torchvision import transforms
from torchvision.models import vgg16, VGG16_Weights
import torch
import torch.nn.functional as F
import torch.optim as optim
from tqdm import tqdm

In [171]:
labels_df = pd.read_csv("./dataset/labels.csv")
print(labels_df.info())
labels_df.apply(lambda x: len(x.unique()))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10222 entries, 0 to 10221
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      10222 non-null  object
 1   breed   10222 non-null  object
dtypes: object(2)
memory usage: 159.8+ KB
None


id       10222
breed      120
dtype: int64

In [172]:
import gc
del labels_df
gc.collect()

4650

In [173]:
labels = pd.read_csv('./dataset/labels.csv', index_col='id')
labels.sample(5)

,breed
id,
a0bdc9dbefdfa54b40423728f1f3627e,cairn
56ddc4e4e63361693d88bdcb0a5cc3ae,saluki
84e567b15311f0c891858f56f0175867,vizsla
29434945d64aed89e7c1968de3b4285b,malinois
e485830e23ef663878896f45f85751f4,dhole


In [174]:
labels['breed'] = labels['breed'].astype('category').cat.codes

In [175]:
labels.sample(5)

,breed
id,
6f68ebaf89c39021195ce96796fc83ae,109
e8827c1b0698e56e60f735b142d69d18,119
2434d734f7ca85eb4733884ba23b7c68,82
f5091fcbb9e639158ad406a3d793ff62,51
e87a86f29c8709e81bbea9c8139d739c,25


In [176]:
device = torch.device('cpu')
if torch.backends.mps.is_available():
    device = torch.device('mps')
elif torch.cuda.is_available():
    device = torch.device('cuda')

device

device(type='mps')

In [177]:
batch_size = 16
num_epochs = 10
learning_rate = 0.001

In [178]:
class DogImgDataset(Dataset):
    def __init__(self, dir_name, labels):
        self.files = os.listdir(dir_name)
        self.img_paths = [os.path.join(dir_name, file) for file in self.files]
        self.transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=0.5, std=0.5)
        ])
        self.labels = labels

    def __getitem__(self, index):
        path = self.img_paths[index]
        index = path.split("/")[-1].split(".")[0]
        label = self.labels.loc[index, 'breed']

        image = Image.open(path)
        image = self.transform(image)
        return image, torch.tensor(label)

    def __len__(self):
        return len(self.img_paths)
    
dataset = DogImgDataset("dataset/train", labels)
train_set, validate_set = random_split(dataset, [0.8, 0.2], generator=torch.Generator().manual_seed(42))
train_loader = DataLoader(train_set, batch_size=16, shuffle=True)
validate_loader = DataLoader(validate_set, batch_size=16, shuffle=True)
dataset[0]

(tensor([[[-0.7569, -0.8510, -0.7882,  ..., -0.8745, -0.8431, -0.8353],
          [-0.7647, -0.8510, -0.7882,  ..., -0.8510, -0.8824, -0.8902],
          [-0.7647, -0.8431, -0.7961,  ..., -0.8353, -0.8588, -0.8667],
          ...,
          [-0.8118, -0.8118, -0.8118,  ..., -0.0353, -0.0431, -0.0824],
          [-0.7647, -0.7725, -0.7725,  ..., -0.0510, -0.0902, -0.1216],
          [-0.7412, -0.7490, -0.7490,  ..., -0.0353, -0.1059, -0.1373]],
 
         [[-0.7961, -0.8980, -0.8431,  ..., -0.8980, -0.8824, -0.8745],
          [-0.8039, -0.8902, -0.8510,  ..., -0.8745, -0.9137, -0.9294],
          [-0.8039, -0.8824, -0.8588,  ..., -0.8588, -0.8980, -0.9059],
          ...,
          [-0.8510, -0.8510, -0.8588,  ..., -0.0118, -0.0196, -0.0588],
          [-0.8431, -0.8353, -0.8353,  ..., -0.0275, -0.0667, -0.0980],
          [-0.8353, -0.8275, -0.8275,  ..., -0.0118, -0.0824, -0.1137]],
 
         [[-0.8431, -0.9451, -0.8902,  ..., -0.9373, -0.9059, -0.8980],
          [-0.8510, -0.9373,

In [179]:
model = vgg16(pretrained=True)
model.classifier = nn.Sequential(
    nn.Linear(512 * 7 * 7, 4096, bias=True),
    nn.ReLU(inplace=True),
    nn.Dropout(p=0.5, inplace=False),
    nn.Linear(4096, 4096, bias=True),
    nn.ReLU(inplace=True),
    nn.Dropout(p=0.5, inplace=False),
    nn.Linear(4096, 120, bias=True),
)
model.to(device=device)

/Users/zhixiang.chenczx/tasks/kaggle/.venv/lib/python3.13/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/zhixiang.chenczx/tasks/kaggle/.venv/lib/python3.13/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [180]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
from IPython.display import clear_output

model.train()
for epoch in range(num_epochs):
    for batch_idx, (data, targets) in enumerate(tqdm(train_loader)):
        # Get data to cuda if possible
        data = data.to(device=device)
        targets = targets.to(device=device)
        optimizer.zero_grad()
        # forward
        scores = model(data)
        loss = criterion(scores, targets)
        loss.backward()
        optimizer.step()
        if batch_idx != 0 and batch_idx % 100 == 0:
            clear_output(wait=True)
            print(f"Epoch {epoch}/{num_epochs} Batch {batch_idx}/{len(train_loader)} Loss {loss.item():.4f}")
            # save model
    print(f"Epoch {epoch}/{num_epochs} Loss {loss.item():.4f}")

 20%|█▉        | 101/512 [01:00<04:59,  1.37it/s]

Epoch 0/10 Batch 100/512 Loss 4.7907


 36%|███▌      | 185/512 [02:00<04:30,  1.21it/s]